# Практическое задание №3 по теме "Связь бизнес-показателей и DS-метрик".

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.sample(10, random_state=12)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
26505,37865,16564,1,157,54.0,90,60,1,1,0,0,1,0
16099,22984,20310,1,158,70.0,120,80,3,3,0,0,1,0
35596,50835,21738,1,169,82.0,140,90,1,1,1,0,1,1
62735,89583,22808,2,170,78.0,140,80,1,1,1,0,0,1
67323,96126,19057,1,155,100.0,150,90,3,1,0,0,0,1
69982,99973,18941,1,153,86.0,130,90,1,2,0,0,1,1
49905,71239,19159,1,165,62.0,120,70,1,1,0,0,0,0
12076,17249,16837,1,155,73.0,120,80,1,2,0,0,1,0
59726,85267,17214,2,165,54.0,140,90,1,1,0,0,1,1
10726,15335,23435,1,162,82.0,140,90,1,1,0,0,1,0


In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), df['cardio'], random_state=0)

In [4]:
# выбор колонки (признака)
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

# выбрать колонку с числовыми данными
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

# классическое преобразование категорийных признаков
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler

# отобрали числовые, категорийные и бинарные признаки
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

# списки пайплайнов для каждого признака
continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [6]:
result_dict = {'LogReg':[], 'XGB':[], 'kNN':[]}

## Задание №1, 2

#### 1. Logistic Regression

In [7]:
%%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: total: 1.84 s
Wall time: 1.85 s


In [8]:
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.7867401104915408+-0.00852135511666111


In [9]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
ix = np.argmax(fscore)
ras = roc_auc_score(y_test, y_score)

result_dict['LogReg'].append(fscore[ix])
result_dict['LogReg'].append(precision[ix])
result_dict['LogReg'].append(recall[ix])
result_dict['LogReg'].append(ras)

print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838


#### 2. Xgboost

In [10]:
%%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', xgboost.XGBClassifier(n_estimators=100, random_state=0)),
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: total: 58.4 s
Wall time: 4.91 s


In [11]:
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.7962156108988848+-0.004388386094897168


In [12]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
ix = np.argmax(fscore)
ras = roc_auc_score(y_test, y_score)

result_dict['XGB'].append(fscore[ix])
result_dict['XGB'].append(precision[ix])
result_dict['XGB'].append(recall[ix])
result_dict['XGB'].append(ras)

print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.347103, F-Score=0.738, Precision=0.665, Recall=0.828


#### 3. k-NN

In [13]:
%%time
classifier = Pipeline([
    ('features',feats),
    ('classifier', KNeighborsClassifier(n_neighbors = 5))
])

cv_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)

classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CPU times: total: 11.2 s
Wall time: 10.4 s


In [14]:
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.6894341449677974+-0.0036001530814609166


In [15]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
ix = np.argmax(fscore)
ras = roc_auc_score(y_test, y_score)

result_dict['kNN'].append(fscore[ix])
result_dict['kNN'].append(precision[ix])
result_dict['kNN'].append(recall[ix])
result_dict['kNN'].append(ras)

print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.400000, F-Score=0.686, Precision=0.588, Recall=0.822


In [16]:
result_dict

{'LogReg': [0.7303229695112764,
  0.6474307596402173,
  0.8375576036866359,
  0.7840347790421852],
 'XGB': [0.7378899835796389,
  0.6652479644707624,
  0.8283410138248848,
  0.797227760535858],
 'kNN': [0.6858626159688507,
  0.5884682009403613,
  0.821889400921659,
  0.6940984174007545]}

## Задание №3

In [17]:
result_df = pd.DataFrame(result_dict, index=['F-score', 'Precision', 'Recall', 'Roc Auc Score'])
result_df = result_df.T
result_df

,F-score,Precision,Recall,Roc Auc Score
LogReg,0.730323,0.647431,0.837558,0.784035
XGB,0.737890,0.665248,0.828341,0.797228
kNN,0.685863,0.588468,0.821889,0.694098


## Задание №4

Из полученных результатов можно сделать вывод, что худшие результаты по всем метрикам у алгоритма **k-NN**. Алгоритмы **Logistic Regression** и **XGboost** показали примерно одинаковые результаты, но **XGboost** все-таки немного лучше.

Однако, важно обратить внимание на то, что по времени обработки **XGboost** сильно проиграл **Logistic Regression**.

## Задание №5

В лекции преподаватель несколько раз упоминал, что метрика *roc_auc_curve* совершенно не подходит для датасетов, в которых присутствует сильный дисбаланс классов. В таких случаях лучше всего использовать метрику **F-score** и соответственно *precision_recall_curve*.

В задании предлагается рассмотреть пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

F-score:

$$precision = \frac{TP}{TP+FP}$$

$$recall = \frac{TP}{TP+FN}$$

$$Fscore = (1+\beta^2)\cdot\frac{precision \cdot recall}{(\beta^2 \cdot precision)+recall}$$

Roc Auc score:

$$TPR = \frac{TP}{TP+FN}$$

$$FPR = \frac{FP}{FP+TN}$$

Первая модель

In [18]:
y_true = [1]*100 + [0]*99900
y_score = [1]*90 + [0]*10 + [1]*10 + [0]*99890

In [19]:
cnf_matrix = confusion_matrix(y_true, y_score)
TN = cnf_matrix[0][0]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]
FP = cnf_matrix[0][1]
precision = TP/(TP+FP)
recall = TP/(TP+FN)
beta = 1
f_score = 2*precision*recall/(precision + recall)
ras = roc_auc_score(y_true, y_score)
print('F-Score=%.3f, \nPrecision=%.3f, \nRecall=%.3f, \nRoc Auc Score=%.3f' % (f_score, precision, recall, ras))

F-Score=0.900, 
Precision=0.900, 
Recall=0.900, 
Roc Auc Score=0.950


Вторая модель

In [20]:
y_true = [1]*100 + [0]*99900
y_score = [1]*90 + [0]*10 + [1]*910 + [0]*98990

In [21]:
cnf_matrix = confusion_matrix(y_true, y_score)
TN = cnf_matrix[0][0]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]
FP = cnf_matrix[0][1]
precision = TP/(TP+FP)
recall = TP/(TP+FN)
beta = 1
f_score = 2*precision*recall/(precision + recall)
ras = roc_auc_score(y_true, y_score)
print('F-Score=%.3f, \nPrecision=%.3f, \nRecall=%.3f, \nRoc Auc Score=%.3f' % (f_score, precision, recall, ras))

F-Score=0.164, 
Precision=0.090, 
Recall=0.900, 
Roc Auc Score=0.945


**Вывод:**

Рассмотрим результаты двух примеров. В первом случае нам повезло и несбалансированность выборки не так сильно заметно, потому что модель определила правильно классы примерно в той же доле, что и разбиты классы в выборке. Однако, во втором случае, мы видим что модель предсказала очень много FP классов, что привело к сильному снижению точности и соответственно F-score. 

Если теперь судить по метрикам при несбалансированности классов, то определенно предпочтительней использовать F-score, который улавливает FP ошибки и вовремя сигнализирует нам о том, что с моделью что-то не так. Roc Auc Score вообще практически никак не отриагировал на появление ошибок и поэтому, если мы будем использовать его, то мы можем и не узнать, что у нас проблемы.

Определенно первая модель намного качественнее, поскольку показатель F-score у нее выше.

---